In [1]:
import tensorflow as tf
from tensorflow import keras
from utils import *
import struct
import numpy as np
from model_define import MyModel,NMSE_loss

In [2]:
###########################以下仅为信道数据载入和链路使用范例############
data1=open('H.bin','rb')
H1=struct.unpack('f'*2*2*2*32*320000,data1.read(4*2*2*2*32*320000))
H1=np.reshape(H1,[320000,2,4,32])
H=H1[:,1,:,:]+1j*H1[:,0,:,:]

Htest=H[300000:,:,:]
H=H[:300000,:,:]

In [3]:
def LMMSE(H_e,y_e):
    a = H_e*H_e.H + 1/10*np.mat(np.eye(2))
    W = H_e.H*a.I
    return (W*y_e).reshape(2)
def ZF(H_e,y_e):
    return (H_e.I*y_e).reshape(2)
def deModulation(Q):
    Qr=np.real(Q)
    Qi=np.imag(Q)
    bits=np.zeros([Q.shape[0],2])
    bits[:,0]=Qr>0
    bits[:,1]=Qi>0
    return bits.reshape([-1])  # This is just for QAM modulation
def generatorXY(batch, H,SNR):
    input_labels = []
    input_samples = []
    input_H = []
    for row in range(0, batch):
        mode = np.random.randint(0, 3)
        #mode = 0
        SNRdb = np.random.randint(0, 5)+SNR
        bits0 = np.random.binomial(n=1, p=0.5, size=(128 * 4,))
        bits1 = np.random.binomial(n=1, p=0.5, size=(128 * 4,))
        X = [bits0, bits1]
        temp = np.random.randint(0, len(H))
        HH = H[temp]
        YY = MIMO(X, HH, SNRdb, mode, Pilotnum) / 20  ###
        XX = np.concatenate((bits0, bits1), 0)
        input_labels.append(XX)
        input_samples.append(YY)
        input_H.append(HH)
    batch_y = np.asarray(input_samples)
    batch_x = np.asarray(input_labels)
    batch_H = np.asarray(input_H)
    
    Y_mat = np.transpose(np.reshape(batch_y,[batch,256,8]),[0,2,1])
    Y_pilot = Y_mat[:,[0,1,4,5],:].reshape(batch,1024)
    
    return batch_y, batch_x, batch_H,Y_pilot


In [24]:
K = 256
Pilotnum=8
mu = 2

P = Pilotnum * 2
Pilot_file_name = 'Pilot_' + str(P)
bits = np.loadtxt(Pilot_file_name, delimiter=',')
pilotValue = Modulation(bits, mu)
pilotValue0 = pilotValue[0:P:2]
pilotValue1 = pilotValue[1:P:2]

N = 1000
Y, X, HH,Y_pilot = generatorXY(N, H,8)
Y_mat = np.transpose(np.reshape(Y,[N,256,8]),[0,2,1])
Y_antena0 = Y_mat[:,0:4,:]
Y_antena0_pilot = Y_antena0[:,0,:]+1j*Y_antena0[:,1,:]

Y_antena1 = Y_mat[:,4:8,:]
Y_antena1_pilot = Y_antena1[:,0,:]+1j*Y_antena1[:,1,:]

Y_antena0_data = Y_antena0[:,2,:]+1j*Y_antena0[:,3,:]
Y_antena1_data = Y_antena1[:,2,:]+1j*Y_antena1[:,3,:]

y = np.zeros([N,K,2]).astype(complex)
y[:,:,0] = Y_antena0_data
y[:,:,1] = Y_antena1_data

model_input = keras.Input(shape=(1024))
model_output = MyModel(model_input)
model = keras.Model(inputs=model_input, outputs=model_output)

#model.summary()
model.compile(optimizer='adam', loss=NMSE_loss)

model.load_weights(".model_"+str(Pilotnum)+".best.h5")

out_H = model.predict(Y_pilot, batch_size=128)/20
H_est = out_H[:,:,:,0]+1j*out_H[:,:,:,1]

def NMSE(x, x_hat):
    x = np.reshape(x[:, :, :], (len(x), -1))
    x_hat = np.reshape(x_hat[:, :, :], (len(x_hat), -1))
    
    power = np.sum(abs(x) ** 2, axis=1)
    mse = np.sum(abs(x - x_hat) ** 2, axis=1)
    nmse = np.mean(mse / power)
    return nmse

print('the NMSE is ',NMSE(HH/20, H_est))

H_est00_t = np.hstack((H_est[:,0,:],np.zeros([N,K-32])))
H_est00 = np.fft.fft(H_est00_t)
H_est01_t = np.hstack((H_est[:,1,:],np.zeros([N,K-32])))
H_est01 = np.fft.fft(H_est01_t)
H_est10_t = np.hstack((H_est[:,2,:],np.zeros([N,K-32])))
H_est10 = np.fft.fft(H_est10_t)
H_est11_t = np.hstack((H_est[:,3,:],np.zeros([N,K-32])))
H_est11 = np.fft.fft(H_est11_t)

H_MIMO = np.zeros([N,K,2,2]).astype(complex)
H_MIMO[:,:,0,0] = H_est00
H_MIMO[:,:,1,0] = H_est01
H_MIMO[:,:,0,1] = H_est10
H_MIMO[:,:,1,1] = H_est11

def LMMSE(H_e,y_e):
    a = H_e*H_e.H + 1/10*np.mat(np.eye(2))
    W = H_e.H*a.I
    return (W*y_e).reshape(2)
def ZF(H_e,y_e):
    return (H_e.I*y_e).reshape(2)
x = np.zeros([N,K,2]).astype(complex)

for i in range(N):
    for j in range(K):
        H_e = np.mat(H_MIMO[i,j,:,:])
        y_e = np.mat(y[i,j,:]).T
        x[i,j,:] = LMMSE(H_e,y_e)

def deModulation(Q):
    Qr=np.real(Q)
    Qi=np.imag(Q)
    bits=np.zeros([Q.shape[0],2])
    bits[:,0]=Qr>0
    bits[:,1]=Qi>0
    return bits.reshape([-1])  # This is just for QAM modulation

X_est = np.zeros([N,1024])
for i in range(N):
    X_est[i,0:512] = deModulation(x[i,:,0])
    X_est[i,512:1024] = deModulation(x[i,:,1])
accuracy = np.sum(X_est == X)/N/1024
print('the accuracy is ',accuracy)

the NMSE is  0.35421825018922054
the accuracy is  0.7998544921875


In [25]:
K = 256
Pilotnum=32
mu = 2

P = Pilotnum * 2
Pilot_file_name = 'Pilot_' + str(P)
bits = np.loadtxt(Pilot_file_name, delimiter=',')
pilotValue = Modulation(bits, mu)
pilotValue0 = pilotValue[0:P:2]
pilotValue1 = pilotValue[1:P:2]

N = 1000
Y, X, HH,Y_pilot = generatorXY(N, H,8)
Y_mat = np.transpose(np.reshape(Y,[N,256,8]),[0,2,1])
Y_antena0 = Y_mat[:,0:4,:]
Y_antena0_pilot = Y_antena0[:,0,:]+1j*Y_antena0[:,1,:]

Y_antena1 = Y_mat[:,4:8,:]
Y_antena1_pilot = Y_antena1[:,0,:]+1j*Y_antena1[:,1,:]

Y_antena0_data = Y_antena0[:,2,:]+1j*Y_antena0[:,3,:]
Y_antena1_data = Y_antena1[:,2,:]+1j*Y_antena1[:,3,:]

y = np.zeros([N,K,2]).astype(complex)
y[:,:,0] = Y_antena0_data
y[:,:,1] = Y_antena1_data

model_input = keras.Input(shape=(1024))
model_output = MyModel(model_input)
model = keras.Model(inputs=model_input, outputs=model_output)

#model.summary()
model.compile(optimizer='adam', loss=NMSE_loss)

model.load_weights(".model_"+str(Pilotnum)+".best.h5")

out_H = model.predict(Y_pilot, batch_size=128)/20
H_est = out_H[:,:,:,0]+1j*out_H[:,:,:,1]

def NMSE(x, x_hat):
    x = np.reshape(x[:, :, :], (len(x), -1))
    x_hat = np.reshape(x_hat[:, :, :], (len(x_hat), -1))
    
    power = np.sum(abs(x) ** 2, axis=1)
    mse = np.sum(abs(x - x_hat) ** 2, axis=1)
    nmse = np.mean(mse / power)
    return nmse

print('the NMSE is ',NMSE(HH/20, H_est))

H_est00_t = np.hstack((H_est[:,0,:],np.zeros([N,K-32])))
H_est00 = np.fft.fft(H_est00_t)
H_est01_t = np.hstack((H_est[:,1,:],np.zeros([N,K-32])))
H_est01 = np.fft.fft(H_est01_t)
H_est10_t = np.hstack((H_est[:,2,:],np.zeros([N,K-32])))
H_est10 = np.fft.fft(H_est10_t)
H_est11_t = np.hstack((H_est[:,3,:],np.zeros([N,K-32])))
H_est11 = np.fft.fft(H_est11_t)

H_MIMO = np.zeros([N,K,2,2]).astype(complex)
H_MIMO[:,:,0,0] = H_est00
H_MIMO[:,:,1,0] = H_est01
H_MIMO[:,:,0,1] = H_est10
H_MIMO[:,:,1,1] = H_est11

def LMMSE(H_e,y_e):
    a = H_e*H_e.H + 1/10*np.mat(np.eye(2))
    W = H_e.H*a.I
    return (W*y_e).reshape(2)
def ZF(H_e,y_e):
    return (H_e.I*y_e).reshape(2)
x = np.zeros([N,K,2]).astype(complex)

for i in range(N):
    for j in range(K):
        H_e = np.mat(H_MIMO[i,j,:,:])
        y_e = np.mat(y[i,j,:]).T
        x[i,j,:] = LMMSE(H_e,y_e)

def deModulation(Q):
    Qr=np.real(Q)
    Qi=np.imag(Q)
    bits=np.zeros([Q.shape[0],2])
    bits[:,0]=Qr>0
    bits[:,1]=Qi>0
    return bits.reshape([-1])  # This is just for QAM modulation

X_est = np.zeros([N,1024])
for i in range(N):
    X_est[i,0:512] = deModulation(x[i,:,0])
    X_est[i,512:1024] = deModulation(x[i,:,1])
accuracy = np.sum(X_est == X)/N/1024
print('the accuracy is ',accuracy)

the NMSE is  0.11793507393035878
the accuracy is  0.8775400390625


In [23]:
Y_pilot.shape

(1000, 1024)